In [2]:
Sys.setenv(LANG = "en")
# clear workspace
# rm(list=ls())

library(biomaRt)
library(clusterProfiler)
library(DOSE)
library(dplyr)
library(httr)
library(igraph)
library(illuminaHumanv4.db)
library(limma)
library(PKNCA)
# library(rstudioapi)
library(rWikiPathways)
library(RCy3)
library(SPARQL)
library(stringr)
library(tidyr)


In [3]:
# directories to be used
BASE_DIR <- file.path(getwd(), "..")
DATA_DIR <- file.path(BASE_DIR, "data")
R_DIR <- file.path(BASE_DIR, "R")

In [33]:
# functions to be used
source(file.path(R_DIR, "functions.R"))

In [4]:
# loading gene expression data
neuro_limma <- read.csv(file.path(DATA_DIR, "neuro_limma.csv"))
nonneuro_limma <- read.csv(file.path(DATA_DIR, "nonneuro_limma.csv"))
neuro_limma$entrez_id <- as.character(neuro_limma$entrez_id)
nonneuro_limma$entrez_id <- as.character(nonneuro_limma$entrez_id)

In [5]:
head(neuro_limma, 3)

,entrez_id,logFC,AveExpr,t,P.Value,adj.P.Val,B,ensembl_id,hgnc_symbol,weight
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
1,10,-0.02629681,-0.01138144,-0.8271803,0.4103833,0.9999456,-5.420949,ENSG00000156006,NAT2,0.9872027
2,100,0.01877344,-0.03663463,0.2723318,0.7860080,0.9999456,-5.696260,ENSG00000196839,ADA,0.9957447
3,1000,0.03458245,-0.01009304,0.7138777,0.4772012,0.9999456,-5.499530,ENSG00000170558,CDH2,0.9897710


In [6]:
head(nonneuro_limma, 3)

,entrez_id,logFC,AveExpr,t,P.Value,adj.P.Val,B,ensembl_id,hgnc_symbol,weight
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
1,10,0.04181197,0.006603150,1.455833,0.148712093,0.8113781,-4.876509,ENSG00000156006,NAT2,0.9564089
2,100,0.15873173,0.004253595,2.693407,0.008352143,0.5797387,-2.613519,ENSG00000196839,ADA,0.6486671
3,1000,0.04920319,-0.003537535,1.184004,0.239344386,0.8526065,-5.202980,ENSG00000170558,CDH2,0.9742188


In [20]:
# WikiPathways from .gmt
wp2gene <- clusterProfiler::read.gmt(file.path(DATA_DIR, "wikipathways-20201110-gmt-Homo_sapiens.gmt"))
wp2gene <- wp2gene %>% tidyr::separate(term, c("name","version","wpid","org"), "%")
wpid2gene <- wp2gene %>% dplyr::select(wpid,gene) #TERM2GENE
wpid2name <- unique(wp2gene %>% dplyr::select(wpid,name)) #TERM2NAME

# wp_pathways <- sapply(wp_files, getSplit)
# names(wp_pathways) <- NULL

In [22]:
# If want to combine reactome information
source(file.path(R_DIR, "getReactome.R"))
wpid2gene <- rbind(wpid2gene, reactome2gene)
wpid2name <- rbind(wpid2name, reactome2name)

[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
    wpid  gene
1 WP1929  2147
2 WP1929  5595
3 WP1929  5594
4 WP1929  2151
5 WP1929  2793
6 WP1929 10681
      wpid                                                              name
1   WP1929 Thrombin signalling through proteinase activated receptors (PARs)
32  WP2761             MyD88:MAL(TIRAP) cascade initiated on plasma membrane
67  WP2768                     MyD88 dependent cascade initiated on endosome
98  WP3549                                                         Mitophagy
125 WP1788                                Bile acid and bile salt metabolism
167 WP1866                             NCAM signaling for neurite out-growth


In [27]:
# create graph object, you need limma_result as an argument
limma_result <- nonneuro_limma
source(file.path(R_DIR, "create_graph.R"))
nonneuro_g <- g
nonneuro_merged_edge <- merged_edge
nonneuro_merged_node <- merged_node

In [29]:
limma_result <- neuro_limma
source(file.path(R_DIR, "create_graph.R"))
neuro_g <- g
neuro_merged_edge <- merged_edge
neuro_merged_node <- merged_node

In [36]:
# (OPTIONAL): Crude run of pathway interaction 
g <- neuro_g
source(file.path(R_DIR, "crude_run_pathway_interaction.R"))
neuro_full_paths_list <- full_paths_list

[1] "We are at: 0"
[1] "We are at: 100"


Warning message in shortest_paths(subg, one_pathway_node, another_pathway_node, :
“At structural_properties.c:4597 :Couldn't reach some vertices”


[1] "We are at: 200"
[1] "We are at: 300"
[1] "We are at: 400"
[1] "We are at: 500"
[1] "We are at: 600"
[1] "We are at: 700"


Warning message in shortest_paths(subg, one_pathway_node, another_pathway_node, :
“At structural_properties.c:4597 :Couldn't reach some vertices”


[1] "We are at: 800"


Warning message in shortest_paths(subg, one_pathway_node, another_pathway_node, :
“At structural_properties.c:4597 :Couldn't reach some vertices”


[1] "We are at: 900"
[1] "We are at: 1000"


Warning message in shortest_paths(subg, one_pathway_node, another_pathway_node, :
“At structural_properties.c:4597 :Couldn't reach some vertices”


[1] "We are at: 1100"


Warning message in shortest_paths(subg, one_pathway_node, another_pathway_node, :
“At structural_properties.c:4597 :Couldn't reach some vertices”


In [39]:
# sel_wp <- subset(neuro_pval_df, pvalue<0.05)$wpid
# sel_wp denotes a vector of wpids.
# can use either p-value or random lists as below
sel_wp <- wpid2name$wpid[1:50]
g <- neuro_g
source(file.path(R_DIR, "collect_final_result.R"))
neuro_sel_path_df <- sel_path_df

Warning message in shortest_paths(subg, one_pathway_node, another_pathway_node, :
“At structural_properties.c:4597 :Couldn't reach some vertices”


In [41]:
head(sel_path_df)

source,target
<chr>,<chr>
WP4657,1399
1399,5159
5159,1901
1901,WP117
WP4657,1399
1399,5159
